In [31]:
import numpy as np
import pandas as pd

In [32]:
def dateTime(row):
    date = row['time'].split("-")[1].strip()
    time = row['time'].split("-")[2].split(" ")[1].strip()
    return date + " " + time

In [33]:
def fixClouds(df,row):
    if pd.isnull(row['clouds']):
        if row['station'] == "Bratislava - Mlynská Dolina":
            try:
                ba = df[(df['time']==row['time']) & (df['station'] == "Bratislava Koliba")]['clouds'].values[0]
            except:
                return "Jasno"
            if pd.isnull(ba):
                return "Jasno"
            return ba
        else:
            return "Jasno"
    else:
        return row['clouds']

In [39]:
df1 = pd.read_csv("weather.csv")
df2 = pd.read_csv("data.csv")
df = df1.append(df2, sort=False)

df['time'] = df.apply(lambda x : dateTime(x),axis=True)
df['time'] = pd.to_datetime(df['time'], format="%d.%m.%Y %H:%M")
df = df.drop_duplicates(subset=['time','station'], keep='last')
df['wind_speed'] = df['wind_speed'].apply(lambda x : float(x.split("m/s")[0].strip()))
df['temperature'] = df['temperature'].apply(lambda x : float(x.split("°C")[0].strip()))
#df['weather'] = df['weather'].fillna("pekne")
df['wind_impact'] = df['wind_impact'].fillna("0 m/s")
df['wind_impact'] = df['wind_impact'].apply(lambda x : float(x.split("m/s")[0].strip()))
df['clouds'] = df.apply(lambda x : fixClouds(df,x),axis=True)
df = df.sort_values(by='time') 
df = df.reset_index(drop=True)

In [42]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5246 entries, 0 to 5245
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   time         5246 non-null   object
 1   station      5246 non-null   object
 2   temperature  5246 non-null   object
 3   wind_dir     5246 non-null   object
 4   wind_speed   5246 non-null   object
 5   wind_impact  149 non-null    object
 6   clouds       2199 non-null   object
 7   weather      287 non-null    object
dtypes: object(8)
memory usage: 328.0+ KB


In [30]:
df.to_csv("prediction_dataset.csv")